In [34]:
import requests
import json
class LLMClient:
    def __init__(self, api_key):
        self._url = 'https://np6jbwtoaiv4y8cw.us-east4.gcp.endpoints.huggingface.cloud/v1/chat/completions'
        self._api_key = api_key

    def make_chat_completions_request(self, model, messages, temperature, max_tokens):
        headers = {
            "Authorization": f"Bearer {self._api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": False,
        }
        response = requests.post(self._url, headers=headers, json=payload, timeout=60)
        response.raise_for_status()
        return response.json()

In [ ]:
key = 'your_api_key_here'  # Replace with your actual API key
llmClient = LLMClient(key)

response = llmClient.make_chat_completions_request(
    model="27b-text-it",
    messages=[
        {"role": "user", "content": "Hello, how are you?"},
    ],
    temperature=0.7,
    max_tokens=150
)

print(response)

{'object': 'chat.completion', 'id': '', 'created': 1747532697, 'model': '27b-text-it', 'system_fingerprint': '3.2.3-sha-a1f3ebe', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Hello! I'm doing well, thank you for asking. As an AI, I don't have feelings or a physical state, but I'm functioning optimally and ready to help.\n\nHow about you? How are you doing today?\n"}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 15, 'completion_tokens': 52, 'total_tokens': 67}}


In [36]:
import json

# Load specialties from the JSON file
with open('../services/specialties.json', 'r') as f:
    specialties = json.load(f)

# Extract relevant information for our prompt
specialty_descriptions = [{
    "specialtyName": specialty["specialtyName"],
    "description": specialty["description"]
} for specialty in specialties]

print(f"Loaded {len(specialty_descriptions)} specialties")

Loaded 72 specialties


In [37]:
def create_specialty_matching_prompt(patient_description):
    """Create a prompt for the LLM to match a patient description to a specialty"""
    
    # Construct the prompt with context and schema requirements
    prompt = f"""
    You are a medical professional tasked with determining the most appropriate medical specialty for a patient based on their description of symptoms or conditions.

    Below are the available medical specialties with their descriptions:

    {format_specialties(specialty_descriptions)}

    PATIENT DESCRIPTION:
    {patient_description}

    TASK: Based on the patient's description, determine the most appropriate medical specialty from the list above. 
    Consider the symptoms, conditions, and affected body systems described by the patient.

    Your response "MUST" strictly adhere to the following JSON format:

    Example (not to include ```json ```): {{
    "RECOMMENDED_SPECIALTY": ["specialty1", "specialty2", "specialty3"],
    "REASONING": "Explain why these specialties are recommended based on the patient's symptoms and conditions.",
    "CONFIDENCE": "One of 'High', 'Medium', or 'Low'"
    }}

    - RECOMMENDED_SPECIALTY: array of strings (up to 3 specialties from the provided list, in order of relevance)
    - REASONING: string (detailed but concise explanation of your recommendation, Pretend you are a medical advisor and provide a easy to understand explanation for this patient why these specialties are recommended in first person (maximum 30 words))
    - CONFIDENCE: string (must be one of: "High", "Medium", or "Low")
    """
    
    return prompt


def format_specialties(specialties):
    """Format the specialties list for the prompt"""
    formatted = ""
    for specialty in specialties:
        formatted += f"- {specialty['specialtyName']}: {specialty['description']}\n"
    return formatted

In [38]:
# Example patient descriptions
patient_examples = [
    "I've been experiencing severe joint pain in my knees and fingers for the past few months. The joints are often swollen and stiff, especially in the morning. I've also been feeling unusually tired.",
    
    "I have a rash that appeared suddenly on my face and arms. It's red, itchy, and has small bumps. I've tried over-the-counter creams but nothing helps.",
    
    "I've been having recurring headaches that are accompanied by sensitivity to light and sound. Sometimes I also experience nausea during these episodes, which can last for hours.",
    
    "I've been experiencing chest pain and shortness of breath, especially when I exert myself. I also notice that my ankles have been swelling lately."
]

In [41]:
# Choose one of the examples to try
patient_description = patient_examples[1]  # The joint pain example

# Create the prompt
prompt = create_specialty_matching_prompt(patient_description)


# Print the generated prompt to review
print("\n=== GENERATED PROMPT ===\n")
print(prompt)
print("\n=== END OF PROMPT ===\n")

# Use the LLM client to get a prediction with structured output
response = llmClient.make_chat_completions_request(
    model="27b-text-it",
    messages=[
        {"role": "system", "content": "You are a medical professional helping to match patients to the correct medical specialty based on their symptoms and conditions. Always respond with properly structured JSON as instructed."},
        {"role": "user", "content": prompt},
    ],
    temperature=0.3,  # Lower temperature for more focused/predictable responses
    max_tokens=300,
)


=== GENERATED PROMPT ===


    You are a medical professional tasked with determining the most appropriate medical specialty for a patient based on their description of symptoms or conditions.

    Below are the available medical specialties with their descriptions:

    - Emergency medicine: Emergency medicine specialists take care of patients with critical illnesses or injuries.
- General surgery: General surgeons take care of patients who may need surgery.
- Vascular surgery: Vascular surgeons treat diseases of the circulatory system, other than the brain and heart.
- Dermatology: Dermatologists treat skin conditions.
- Internal medicine: Internists treat diseases of the internal organs that don’t require surgery. They also provide primary care for teenagers, adults, and elderly people.
- Hospice/palliative care: Hospice and palliative care physicians manage pain and other distressing symptoms of serious illnesses. 'Hospice care' is palliative care for patients who are expected to 

In [42]:
# Print the response
print("\n=== LLM RESPONSE ===\n")
if "choices" in response and len(response["choices"]) > 0:
    content = response["choices"][0]["message"]["content"]
    content = content.replace("```json", "").replace("```", "").strip()
    print(content)
    
    # Try to parse the JSON from the response
    import json
    
    try:
        parsed_response = json.loads(content)
        print("\n=== PARSED RESPONSE ===\n")
        print(f"Recommended Specialties: {', '.join(parsed_response.get('RECOMMENDED_SPECIALTY', []))}")
        print(f"Reasoning: {parsed_response.get('REASONING', 'Not provided')}")
        print(f"Confidence: {parsed_response.get('CONFIDENCE', 'Not provided')}")
    except json.JSONDecodeError as e:
        print(f"\nCouldn't parse JSON: {e}")
        print("Raw content:", content)
else:
    print("Unexpected response format:", response)


=== LLM RESPONSE ===

{
  "RECOMMENDED_SPECIALTY": ["Dermatology", "Allergy/Immunology"],
  "REASONING": "I recommend seeing a dermatologist for your rash, as they specialize in skin conditions. An allergist might also help if the rash is allergy-related.",
  "CONFIDENCE": "High"
}

=== PARSED RESPONSE ===

Recommended Specialties: Dermatology, Allergy/Immunology
Reasoning: I recommend seeing a dermatologist for your rash, as they specialize in skin conditions. An allergist might also help if the rash is allergy-related.
Confidence: High


## Using this Approach in Production

The above example demonstrates how to use the LLM to match patient descriptions to medical specialties. In a production environment, you might want to create a more structured function that could be imported and used elsewhere in your application.

In [ ]:
def predict_specialty(llm_client, patient_description, specialties_data=None):
    """Predict the most appropriate medical specialty for a patient description.
    
    Args:
        llm_client (LLMClient): An instance of the LLM client
        patient_description (str): The patient's description of their symptoms or condition
        specialties_data (list, optional): List of specialty dictionaries. If None, will load from file.
        
    Returns:
        dict: A dictionary containing the LLM's prediction, including:
            - recommended_specialties: List of recommended specialties (up to 3)
            - confidence: High/Medium/Low
            - reasoning: Explanation for the recommendation
            - raw_response: The full raw response from the LLM
    """
    # Load specialties if not provided
    if specialties_data is None:
        with open('../services/specialties.json', 'r') as f:
            specialties_data = json.load(f)
            
    # Extract relevant information
    specialty_descriptions = [{
        "specialtyName": specialty["specialtyName"],
        "description": specialty["description"]
    } for specialty in specialties_data]
    
    # Create the prompt
    prompt = create_specialty_matching_prompt(patient_description)
    
    # Define the JSON schema for structured output
    json_schema = {
        "type": "object",
        "properties": {
            "RECOMMENDED_SPECIALTY": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Up to 3 specialties from the provided list, in order of relevance"
            },
            "REASONING": {
                "type": "string",
                "description": "Detailed reasoning for the recommendation"
            },
            "CONFIDENCE": {
                "type": "string",
                "enum": ["High", "Medium", "Low"],
                "description": "Confidence level in the recommendation"
            }
        },
        "required": ["RECOMMENDED_SPECIALTY", "REASONING", "CONFIDENCE"]
    }
    
    # Get prediction from the LLM
    response = llm_client.make_chat_completions_request(
        model="27b-text-it",
        messages=[
            {"role": "system", "content": "You are a medical professional helping to match patients to the correct medical specialty based on their symptoms and conditions. Always respond with properly structured JSON as instructed."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.3,
        max_tokens=300,
        response_format={"type": "json_schema", "schema": json_schema}
    )
    
    # Parse the response 
    if "choices" in response and len(response["choices"]) > 0:
        content = response["choices"][0]["message"]["content"]
        
        # Set up default result
        result = {
            "recommended_specialties": [],
            "confidence": None,
            "reasoning": None,
            "raw_response": content
        }
        
        # Try to parse the JSON response
        try:
            parsed = json.loads(content)
            result["recommended_specialties"] = parsed.get("RECOMMENDED_SPECIALTY", [])
            result["confidence"] = parsed.get("CONFIDENCE")
            result["reasoning"] = parsed.get("REASONING")
        except json.JSONDecodeError:
            result["error"] = "Failed to parse JSON from LLM response"
                
        return result
    else:
        # Handle error case
        return {"error": "Failed to get a valid response from the LLM", "raw_response": response}

# Example usage of the production function
# result = predict_specialty(llmClient, "I've been experiencing joint pain and stiffness, especially in the morning")
# print(f"Recommended specialties: {', '.join(result['recommended_specialties'])}")
# print(f"Confidence: {result['confidence']}")
# print(f"Reasoning: {result['reasoning']}")

In [ ]:
# Test the updated function with a new patient description
test_description = "I've been experiencing severe headaches for the past month, primarily on one side of my head. The pain is throbbing and gets worse with light and sound. Sometimes I also feel nauseous during these episodes."

# Get prediction
result = predict_specialty(llmClient, test_description)

# Display results
print("\n=== PREDICTION RESULTS ===\n")
print(f"Recommended specialties: {', '.join(result['recommended_specialties'])}")
print(f"Confidence: {result['confidence']}")
print(f"Reasoning: {result['reasoning']}")

if 'error' in result:
    print(f"Error: {result['error']}")